#  Model Selection Lab Soln
## Grid Search for *k*-NN

To get us started we have an example that fits a *k*-NN model for the `HotelRevHelpfulness` dataset. It assesses three options:
- whether to use a StandardScaler, MinMaxScaler or no scaler. 
- what <em>k</em> to use for <em>k</em>-NN
- what weighting policy

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import load_digits
from sklearn.pipeline import Pipeline
import pandas as pd

In [ ]:
hotel_rev = pd.read_csv('data/HotelRevHelpfulness.csv')
hotel_rev.head()

In [ ]:
hotel_rev.pop("hotelId")   # get rid of ID feature
hotel_rev.head()
y = hotel_rev.pop('reviewHelpfulness').values
X = hotel_rev.values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=1/2,
                                                    random_state=42)
X_train.shape, X_test.shape

In [ ]:
kNNpipe  = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('kNN', KNeighborsClassifier())])

# Parameters for kNN are prefixed with kNN__
param_grid = {'scaler':[StandardScaler(), MinMaxScaler(),'passthrough'], 
              'kNN__n_neighbors':[1,3,5,7],
              'kNN__weights':['uniform','distance']
             }

In [ ]:
grid_search = GridSearchCV(kNNpipe, param_grid=param_grid, verbose = 1)
grid_search = grid_search.fit(X_train,y_train)

In [ ]:
grid_search.best_params_

### All grid search results
The parameter `cv_results_` gives us access to results on all options tested.  
We store the results in a data frame and print the important information. 

In [ ]:
scores_df = pd.DataFrame(grid_search.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df [['rank_test_score', 'mean_test_score', 'param_kNN__n_neighbors', 
            'param_kNN__weights','param_scaler']]

## Grid Search for Naive Bayes
**Q1**  
Repeat the exercise above to fit a Naive Bayes model.  
Consider the same scaling options and `GaussianNB` and `BernoulliNB` as classifier options. 

In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler

Pipeline similar to above but with `naive_bayes` as the classifier.  
In the `param_grid` `GaussianNB` and `BernoulliNB` are the two options to consider for `naive_bayes`.  

In [ ]:
NBpipe  = Pipeline(steps=[
    ('scaler','passthrough'),
    ('naive_bayes', GaussianNB())])

param_grid = {'scaler':[StandardScaler(), MinMaxScaler(),'passthrough'], 
              'naive_bayes':[GaussianNB(), BernoulliNB()]
             }

In [ ]:
grid_search = GridSearchCV(NBpipe, param_grid=param_grid, cv = 10, verbose = 1)
grid_search = grid_search.fit(X_train,y_train)

And the winners are...

In [ ]:
grid_search.best_params_

In [ ]:
scores_df = pd.DataFrame(grid_search.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df[['rank_test_score','param_naive_bayes','mean_test_score','param_scaler']]

## Grid Search for Decision Trees
**Q2**  
Find the best decision tree model for the `HotelRevHelpfulness` dataset considering  `max_leaf_nodes` and the splitting `criterion`. The splitting `criterion` can be either 'gini' or 'entropy', you can select your own options for `max_leaf_nodes`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

There are no preprocessing steps so there is no need for a pipeline.   
We go with [3,5,10,20,50] as the options for `max_leaf_nodes`.

In [ ]:
tree_grid = {'criterion':['gini','entropy'], 
              'max_leaf_nodes':[3,5,10,20,50],
             }
tree = DecisionTreeClassifier()
tree_search = GridSearchCV(tree, param_grid=tree_grid, cv = 10, verbose = 1)
tree_search = tree_search.fit(X_train,y_train)

The winning parameters are...

In [ ]:
tree_search.best_params_

The main message we can take from looking at all the results is that less leaf nodes is inclined to be better.   
This suggests that bushier trees are inclined to overfit.  

In [ ]:
scores_df = pd.DataFrame(tree_search.cv_results_)
scores_df = scores_df.sort_values(by=['rank_test_score']).reset_index(drop='index')
scores_df[['rank_test_score','param_criterion','mean_test_score','param_max_leaf_nodes']]

## Model selection
**Q3**
Which model would you recommend for this dataset?

It's a toss up between a Decision Tree using the gini splitting criterion and max_leaf_nodes = 5 and a k-NN classifier with *k*=5, uniform weighting and no scaling